In [1]:
import sqlite3

con = sqlite3.connect("../scripts/test_output.db", isolation_level=None)
con.execute("PRAGMA journal_mode=WAL;")
cursor = con.cursor()

In [2]:
cursor.execute("SELECT COUNT(*) FROM mace_output;")
row_count = cursor.fetchone()[0]

In [3]:
import re
from tqdm.notebook import tqdm

cursor.execute("SELECT output FROM mace_output")
err = 0
total = 0
for row in tqdm(range(row_count)):
    total += 1
    output = cursor.fetchone()[0]
    search_result = re.search(r".*INPUT(.*)end of input.*", output, re.DOTALL)
    input_lines = search_result.groups()[0]
    x = set()
    for equation in re.finditer(r"(\d+) \* (\d+) = (\d+)\.", input_lines):
        i, j, k = map(int, equation.groups())
        ind = 13 * i + j
        if ind in x:
            err += 1
            break
        else:
            x.add(ind)

In [4]:
print(err, total)

0 699382


In [5]:
cursor.execute("ALTER TABLE mace_output ADD COLUMN number_of_cells INTEGER")

In [6]:
cursor.execute("SELECT rowid, output FROM mace_output")
insert_cursor = con.cursor()
for row in tqdm(range(row_count)):
    data = cursor.fetchone()
    output = data[1]
    row_id = data[0]
    search_result = re.search(r".*INPUT(.*)end of input.*", output, re.DOTALL)
    input_lines = search_result.groups()[0]
    eq_num = 0
    for equation in re.finditer(r"(\d+) \* (\d+) = (\d+)\.", input_lines):
        eq_num += 1
    insert_cursor.execute(
        "UPDATE mace_output SET number_of_cells = ? WHERE rowid = ?",
        (eq_num, row_id)
    )

In [7]:
cursor.execute("""
SELECT
    number_of_cells,
    AVG(model_exists),
    SUM(model_exists),
    COUNT(*)
FROM mace_output
GROUP BY number_of_cells
ORDER BY number_of_cells
""")
cursor.fetchall()

[(1, 1.0, 4633, 4633),
 (2, 1.0, 3539, 3539),
 (3, 1.0, 3431, 3431),
 (4, 1.0, 4100, 4100),
 (5, 0.9733601970002239, 4348, 4467),
 (6, 0.9865970013630168, 4343, 4402),
 (7, 0.9851005711447728, 3967, 4027),
 (8, 0.9329896907216495, 4163, 4462),
 (9, 0.8127759023630226, 3130, 3851),
 (10, 0.7153104130596092, 2892, 4043),
 (11, 0.4306869600987248, 2094, 4862),
 (12, 0.11887913954146617, 420, 3533),
 (13, 0.026637069922308545, 120, 4505),
 (14, 0.0, 0, 4318),
 (15, 0.0, 0, 4383),
 (16, 0.0, 0, 4983),
 (17, 0.0, 0, 4278),
 (18, 0.0, 0, 4337),
 (19, 0.0, 0, 3963),
 (20, 0.0, 0, 5508),
 (21, 0.0, 0, 4087),
 (22, 0.0, 0, 4321),
 (23, 0.0, 0, 4288),
 (24, 0.0, 0, 3740),
 (25, 0.0, 0, 4989),
 (26, 0.0, 0, 4739),
 (27, 0.0, 0, 4152),
 (28, 0.0, 0, 4580),
 (29, 0.0, 0, 3438),
 (30, 0.0, 0, 4097),
 (31, 0.0, 0, 4391),
 (32, 0.0, 0, 5052),
 (33, 0.0, 0, 3987),
 (34, 0.0, 0, 4689),
 (35, 0.0, 0, 4258),
 (36, 0.0, 0, 4084),
 (37, 0.0, 0, 3620),
 (38, 0.0, 0, 4357),
 (39, 0.0, 0, 4033),
 (40, 0.0, 0, 3